In [1]:
import os, os.path
import numpy as np
import pandas as pd
import setup_analysis as sa
import support_functions as sf
import importlib
importlib.reload(sa)
importlib.reload(sf)


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:721: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'support_functions' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py'>

In [4]:
## 
#   2022-07-01: EDMUNDO, THIS IS THE CELL
##


fkdata = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/dev/fake_data/land_use_transition_baseline_assumption_fake_data.xlsx"
fp_mergetarg = os.path.join(sa.dir_out, "tmp_afolu_vars.csv")



df_mk = pd.read_excel(fkdata, sheet_name = "data_sheet")
fields_id = ["year", "cat"]
fields_dat = [x for x in df_mk.columns if (x not in fields_id + ["scen"])]
# set to long and get transition probabilities
df_mkl = pd.melt(df_mk, id_vars = fields_id, value_vars = fields_dat)
def format_prob(row):
    return "pij_%s_to_%s"%tuple(row)
df_mkl["field"] = np.array(df_mkl[["cat", "variable"]].apply(format_prob, axis = 1))

# read dataset to merge into
df_mergetarg = pd.read_csv(fp_mergetarg)

df_mkw = pd.pivot(df_mkl[["year", "field", "value"]], index = "year", columns = "field", values = "value")
df_mkw.reset_index(drop = False, inplace = True)# = df_mkw.rename(columns = {"year": "time_period"})
df_mkw["time_period"] = df_mkw["year"].replace({2016:0, 2050:35})
fields_id = ["time_period"]
fields_dat = [x for x in df_mkw.columns if (x not in fields_id + ["field", "year"])]
df_mkw = df_mkw[fields_id + fields_dat].reset_index(drop = True)

df_mkw_out = pd.DataFrame({"time_period": list(range(36))})
df_mkw_out = pd.merge(df_mkw_out, df_mkw, how = "left")
df_mkw_out = df_mkw_out.interpolate().transpose().sort_index()
df_mkw_out.reset_index(drop = False, inplace = True)
df_mkw_out = df_mkw_out.rename(columns = {"index": "variable"})
df_mkw_out = df_mkw_out.rename(columns = dict(zip(df_mkw_out.columns, [str(x) for x in df_mkw_out.columns])))

df_mkw_out = pd.merge(df_mergetarg[["subsector", "src", "variable", "time_series_id", "strategy_id", "variable_trajectory_group", "variable_trajectory_group_trajectory_type", "normalize_group", "trajgroup_no_vary_q", "uniform_scaling_q", "min_35", "max_35"]], df_mkw_out).sort_values(by = ["variable"]).reset_index(drop = True)
df_mkw_out = pd.concat([df_mergetarg[~df_mergetarg["0"].isna()], df_mkw_out], axis = 0).sort_values(by = ["subsector", "variable"]).reset_index(drop = True) 
#df_mkw_out.to_csv(os.path.join(sa.dir_out, "tmp_afolu_vars_with_mcmc.csv"), index = None, encoding = "UTF-8")


df_mkw_out2 = df_mkw_out[[str(x) for x in range(36)]].transpose().reset_index(drop = True)
var_fields = list(df_mkw_out["variable"])
df_mkw_out2 = df_mkw_out2.rename(columns = dict(zip([(x) for x in range(len(df_mkw_out2.columns))], var_fields)))
df_mkw_out2["time_period"] = list(range(len(df_mkw_out2)))
df_mkw_out2 = df_mkw_out2[["time_period"] + var_fields]

#df_mkw_out2.to_csv(os.path.join(sa.dir_out, "fake_data_afolu.csv"), index = None, encoding = "UTF-8")

In [12]:
def format_markov_matrix_as_field(
    mat: np.ndarray,
    key_vals: list
) -> pd.DataFrame:
    
df_mk = pd.read_excel(fkdata, sheet_name = "data_sheet")
fields_id = ["year", "cat"]
fields_dat = [x for x in df_mk.columns if (x not in fields_id + ["scen"])]
# set to long and get transition probabilities
df_mkl = pd.melt(df_mk, id_vars = fields_id, value_vars = fields_dat)
def format_prob(row):
    return "pij_%s_to_%s"%tuple(row)
df_mkl["field"] = np.array(df_mkl[["cat", "variable"]].apply(format_prob, axis = 1))

pd.melt(df_mk[fields_dat + ["cat"]], id_vars = ["cat"], value_vars = fields_dat)


,cat,variable,value
0,forests_primary,forests_primary,0.996828
1,forests_secondary,forests_primary,0.001000
2,forests_mangroves,forests_primary,0.000000
3,croplands,forests_primary,0.000000
4,grasslands,forests_primary,0.000000
...,...,...,...
123,croplands,other,0.005276
124,grasslands,other,0.002645
125,settlements,other,0.001739
126,wetlands,other,0.009665


In [276]:
df_mkw_out = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "tmp_afolu_vars_with_mcmc_new.csv"))
df_mkw_out2 = df_mkw_out[[str(x) for x in range(36)]].transpose().reset_index(drop = True)
var_fields = list(df_mkw_out["variable"])
df_mkw_out2 = df_mkw_out2.rename(columns = dict(zip([(x) for x in range(len(df_mkw_out2.columns))], var_fields)))
df_mkw_out2["time_period"] = list(range(len(df_mkw_out2)))
var_fields.sort()
df_mkw_out2 = df_mkw_out2[["time_period"] + var_fields]

df_mkw_out2.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_afolu.csv"), index = None, encoding = "UTF-8")

In [277]:
df_mkw_out = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "tmp_afolu_vars_with_mcmc_new_high_yields.csv"))
df_mkw_out2 = df_mkw_out[[str(x) for x in range(36)]].transpose().reset_index(drop = True)
var_fields = list(df_mkw_out["variable"])
df_mkw_out2 = df_mkw_out2.rename(columns = dict(zip([(x) for x in range(len(df_mkw_out2.columns))], var_fields)))
df_mkw_out2["time_period"] = list(range(len(df_mkw_out2)))
var_fields.sort()
df_mkw_out2 = df_mkw_out2[["time_period"] + var_fields]

df_mkw_out2.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_afolu-high_yields.csv"), index = None, encoding = "UTF-8")

In [264]:
#
# 2021-03-04: UPDATE INPUT TO SPLIT OUT EMISSION FACTORS 
#
#

df_mkw_out = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "tmp_afolu_vars_with_mcmc.csv"))
repl_co2 = list(df_mkw_out[[("ef_agactivity" in x) and ("co2" in x) for x in list(df_mkw_out["variable"])]]["variable"])
repl_n2o = list(df_mkw_out[[("ef_agactivity" in x) and ("n2o" in x) for x in list(df_mkw_out["variable"])]]["variable"])
repl_ch4 = list(df_mkw_out[[("ef_agactivity" in x) and ("ch4" in x) for x in list(df_mkw_out["variable"])]]["variable"])
dict_repl = dict([(x, x.replace("_agactivity_", "_soilcarbon_")) for x in repl_co2])
dict_repl.update(dict([(x, x.replace("_agactivity_", "_biomassburning_")) for x in repl_n2o]))
dict_repl.update(dict([(x, x.replace("_agactivity_", "_anaerobicdom_")) for x in repl_ch4]))

# 
df_mkw_out_app = df_mkw_out[df_mkw_out["variable"].isin(repl_co2)].copy()
df_mkw_out_app["variable"].replace(dict([(x, x.replace("_agactivity_", "_biomassdom_").replace("_ha", "_tonne")) for x in repl_co2]), inplace = True)
df_mkw_out_app["src"] = "made_up"


flds_ow = [str(x)for x in range(36)]
df_new = pd.DataFrame(np.array(df_mkw_out_app[flds_ow])/100, columns = flds_ow)
df_mkw_out_app.drop(flds_ow, inplace = True, axis = 1)
df_mkw_out_app = pd.concat([df_mkw_out_app.reset_index(drop = True), df_new.reset_index(drop = True)], axis = 1).reset_index(drop = True)


df_mkw_out["variable"].replace(dict_repl, inplace = True)
df_mkw_out = pd.concat([df_mkw_out, df_mkw_out_app[df_mkw_out.columns]], axis = 0).reset_index(drop = True).sort_values(by = ["subsector", "variable"])
df_mkw_out.to_csv(os.path.join(sa.dir_ref, "fake_data", "tmp_afolu_vars_with_mcmc_new.csv"), index = None, encoding = "UTF-8")


In [267]:
df_mkw_out[[("_biomassdom_" in x) and ("co2" in x) for x in list(df_mkw_out["variable"])]]["variable"]

24    ef_biomassdom_bevs_and_spices_gg_co2_tonne
25            ef_biomassdom_cereals_gg_co2_tonne
26             ef_biomassdom_coffee_gg_co2_tonne
27             ef_biomassdom_fibers_gg_co2_tonne
28             ef_biomassdom_fruits_gg_co2_tonne
29              ef_biomassdom_maize_gg_co2_tonne
30               ef_biomassdom_nuts_gg_co2_tonne
31        ef_biomassdom_oil_bearing_gg_co2_tonne
32             ef_biomassdom_pulses_gg_co2_tonne
33               ef_biomassdom_rice_gg_co2_tonne
34             ef_biomassdom_tubers_gg_co2_tonne
35         ef_biomassdom_vegetables_gg_co2_tonne
Name: variable, dtype: object

,subsector,src,variable,time_series_id,strategy_id,variable_trajectory_group,variable_trajectory_group_trajectory_type,normalize_group,trajgroup_no_vary_q,uniform_scaling_q,...,26,27,28,29,30,31,32,33,34,35
1,Agriculture,made_up,ef_biomassdom_bevs_and_spices_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.74,7.65,7.56,7.47,7.37,7.28,7.19,7.10,7.00,6.91
4,Agriculture,made_up,ef_biomassdom_cereals_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.54,2.52,2.48,2.45,2.41,2.39,2.36,2.32,2.30,2.27
7,Agriculture,made_up,ef_biomassdom_coffee_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.74,7.65,7.56,7.47,7.37,7.28,7.19,7.10,7.00,6.91
10,Agriculture,made_up,ef_biomassdom_fibers_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.54,2.52,2.48,2.45,2.41,2.39,2.36,2.32,2.30,2.27
13,Agriculture,made_up,ef_biomassdom_fruits_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.77,1.75,1.73,1.71,1.69,1.67,1.64,1.62,1.60,1.58
16,Agriculture,made_up,ef_biomassdom_maize_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.39,3.36,3.32,3.28,3.23,3.19,3.15,3.12,3.07,3.03
19,Agriculture,made_up,ef_biomassdom_nuts_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.41,1.40,1.38,1.36,1.34,1.33,1.31,1.29,1.28,1.26
22,Agriculture,made_up,ef_biomassdom_oil_bearing_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.02,2.98,2.94,2.91,2.87,2.84,2.80,2.76,2.73,2.69
25,Agriculture,made_up,ef_biomassdom_pulses_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.41,1.40,1.38,1.36,1.34,1.33,1.31,1.29,1.28,1.26
28,Agriculture,made_up,ef_biomassdom_rice_gg_co2_tonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:

#rescale
#df_mkw_out_app = df_mkw_out_app[df_mkw_out.columns]



In [ ]:
df_mkw_out = pd.read_csv(os.path.join(sa.dir_out, "tmp_afolu_vars_with_mcmc.csv"))
df_mkw_out2 = df_mkw_out[[str(x) for x in range(36)]].transpose()#.reset_index(drop = True)

In [ ]:
df_mkw_out2